In [1]:
import pandas as pd 
import numpy as np 

In [4]:
! pip install -q -U bitsandbytes
# ! pip install -q -U git+https://github.com/huggingface/transformers.git
! pip install transformers==4.31
! pip install -q -U git+https://github.com/huggingface/peft.git
! pip install -q -U git+https://github.com/huggingface/accelerate.git
! pip install -q datasets
! pip install evaluate
! pip install -qqq trl==0.7.1

  Using cached transformers-4.31.0-py3-none-any.whl.metadata (116 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
Failed to build tokenizers


  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
      creating build\lib.win-amd64-cpython-312\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
      creating build\lib.win-amd64-cpython-312\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
      creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
      copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
      creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
      copying py_src

In [3]:
import time
import torch
#import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random


TypeError: expected string or bytes-like object, got 'NoneType'

In [ ]:



df_train = pd.read_csv("dataset/train_ft.csv")
# df_train = df_train.head(50)
df_val = pd.read_csv("dataset/val_ft.csv")
# df_val = df_val.head(10)
#df = df.drop(columns = ["Unnamed: 0"])
train_data = Dataset.from_pandas(df_train)
val_data = Dataset.from_pandas(df_val)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-chat-hf"

bnb_config = BitsAndBytesConfig(
     load_in_4bit=True,
     bnb_4bit_use_double_quant=True,
     bnb_4bit_quant_type="nf4",
     bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", token=".....")
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():

        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

print(model)

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    # target_modules=["query_key_value"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

OUTPUT_DIR = "ckpts"
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    per_device_train_batch_size=8,    
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True,
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=2060,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

peft_model_path="llama_pred_exp"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

TypeError: expected string or bytes-like object, got 'NoneType'

In [1]:
import pandas as pd

In [3]:
df= pd.read_csv("datasets/train_ft.csv")

In [4]:
df.head()

,Case Name,Input,Output,Label,Count,Decision_Count,text
0,"KAMLESH Vs. UNION OF INDIA THROUGH SECRETARY, ...",30.3.92 after noon regular appoint is made. Mi...,0[ds]7. It is clear from the order of appointm...,0,1253,564,### Instruction:\nForecast the likely verdict ...
1,KANWAR PAL SINGH Vs. THE STATE OF UTTAR PRADESH,raised by the appellant in the written submiss...,1[ds]5. We find the submission of the appellan...,1,4065,2268,"### Instruction:\nFirst, predict whether the a..."
2,Manke Ram Vs. State of Haryana,appellant in this case is that even if the pro...,1[ds]6. Having perused the material on record ...,1,1616,708,### Instruction:\nDetermine the likely decisio...
3,Kr. Jyoti Sarup and Another Vs. Board of Reven...,and by the substantive part of sub-section (1)...,"0[ds]7. We are in agreement with the view, exp...",0,2440,784,### Instruction:\nJudge the probable resolutio...
4,"Commissioner Of Income-Tax, Kerala Vs. Gemini ...",the true costs of trading in the particular ye...,1[ds]That case can have no application to the ...,1,3317,722,### Instruction:\nAssess the case to predict t...


In [5]:
df1= pd.read_csv("datasets/train.csv")

In [6]:
df1.head()

,Case Name,Input,Output,Label,Count,Decision_Count
0,"KAMLESH Vs. UNION OF INDIA THROUGH SECRETARY, ...","S. ABDUL NAZEER, J. 1. In this appeal, appella...",0[ds]7. It is clear from the order of appointm...,0,1253,564
1,KANWAR PAL SINGH Vs. THE STATE OF UTTAR PRADESH,"SANJIV KHANNA, J. 1. Leave granted. 2. Kanwar ...",1[ds]5. We find the submission of the appellan...,1,4065,2268
2,Manke Ram Vs. State of Haryana,"Santosh Hegde, J. 1. The incident which gives ...",1[ds]6. Having perused the material on record ...,1,1616,708
3,Kr. Jyoti Sarup and Another Vs. Board of Reven...,"S.K. Das, J. 1. These two appeals and the conn...","0[ds]7. We are in agreement with the view, exp...",0,2440,784
4,"Commissioner Of Income-Tax, Kerala Vs. Gemini ...","Shah, J. 1. Two persons-Walter and Ramasubramo...",1[ds]That case can have no application to the ...,1,3317,722


In [6]:
df2= pd.read_csv("datasets/test.csv")
df2.shape

(3044, 6)

In [5]:
df2["Input"][0]

'Subba Rao, J.1. These two appeals by special leave are filed against the common judgment of a Full Bench of the High Court of Judicature at Nagpur in writ petitions Nos. 22 of 1955 and 274 of 1955 filed by respondents 1, 3 to 6 herein in the said Court.2. The facts in Appeal No. 370 of 1960 may be stated first. Respondent 1, Seth Balkishan Nathani, was the proprietor and lambardar of Mouza Sonpairi in Tehsil and District Raipur. On January 14, 1947, he executed perpetual pastas in favour of his wife, Yashodabai, since deceased, and respondents 4, 5 and 6 in respect of khudkasht and grass lands of Mouza Sonpairi. In Tabdili Jamabandi of the year 1941347 the said lands were recorded as the Occupancy Tenancy Holdings of the said respondent 4 to 6 and respondent 2, Govindlal Nathani, the legal representative of Yashodabai. The same entry was found in the Jamabandis of the subsequent years. The Madhya Pradesh Abolition of Proprietary Rights (Estates, Mahals, Alienated Lands) Act, l950 (1 o

In [18]:
df2["Label"][78]

np.int64(1)